In [1]:
import shutil
import os

dataset_path = "D:\\Job\\image_retrieval\\data\\COVID-19_Radiography_Dataset"
structured_dataset = "D:\\Job\\image_retrieval\\data\\COVID_Dataset"

In [2]:
from dataset import MedicalDataset, get_transforms
from torch.utils.data import DataLoader, random_split

dataset_root = "D:\\Job\\image_retrieval\\data\\COVID_Dataset"

full_dataset = MedicalDataset(dataset_root, transform=get_transforms())

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [3]:
from config_cnn import Config
from cnn_fpn_rmac import CNN_FPN_RMAC
import torch
import torch.nn as nn

cfg = Config()
cfg.NUM_CLASSES = 4  

model = CNN_FPN_RMAC(cfg)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

d:\App\anaconda3\envs\mir_py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\App\anaconda3\envs\mir_py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
pip show torch

Name: torch
Version: 2.6.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: d:\app\anaconda3\envs\mir_py310\lib\site-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: torchaudio, torchvision
Note: you may need to restart the kernel to use updated packages.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

checkpoint_dir = "D:\\Job\\image_retrieval\\models\\checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
best_val_acc = 0.0

for epoch in range(100):
    model.train()
    train_loss, train_correct = 0.0, 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        train_correct += (outputs.argmax(1) == y).sum().item()

    train_acc = train_correct / len(train_dataset)
    train_loss /= len(train_dataset)

    model.eval()
    val_loss, val_correct = 0.0, 0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)

            val_loss += loss.item() * x.size(0)
            val_correct += (outputs.argmax(1) == y).sum().item()

    val_acc = val_correct / len(val_dataset)
    val_loss /= len(val_dataset)

    print(f"Epoch {epoch+1}/100")
    print(f"Train     Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"Val       Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

    checkpoint_path = os.path.join(
        checkpoint_dir, f"epoch_{epoch+1:02d}_valacc_{val_acc:.4f}.pth"
    )
    torch.save(model.state_dict(), checkpoint_path)
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_path = os.path.join(checkpoint_dir, "best_cnn_model.pth")
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ Saved best model at epoch {epoch+1} with val_acc {val_acc:.4f}")

Epoch 1/100
Train     Loss: 0.6061 | Acc: 0.7628
Val       Loss: 0.4054 | Acc: 0.8488
✅ Saved best model at epoch 1 with val_acc 0.8488
Epoch 2/100
Train     Loss: 0.3599 | Acc: 0.8701
Val       Loss: 0.3944 | Acc: 0.8538
✅ Saved best model at epoch 2 with val_acc 0.8538
Epoch 3/100
Train     Loss: 0.2818 | Acc: 0.8974
Val       Loss: 0.4074 | Acc: 0.8561
✅ Saved best model at epoch 3 with val_acc 0.8561
Epoch 4/100
Train     Loss: 0.2399 | Acc: 0.9137
Val       Loss: 0.3862 | Acc: 0.8651
✅ Saved best model at epoch 4 with val_acc 0.8651
Epoch 5/100
Train     Loss: 0.2025 | Acc: 0.9272
Val       Loss: 0.3145 | Acc: 0.8911
✅ Saved best model at epoch 5 with val_acc 0.8911
Epoch 6/100
Train     Loss: 0.1802 | Acc: 0.9354
Val       Loss: 0.2632 | Acc: 0.9024
✅ Saved best model at epoch 6 with val_acc 0.9024
Epoch 7/100
Train     Loss: 0.1557 | Acc: 0.9442
Val       Loss: 0.2263 | Acc: 0.9166
✅ Saved best model at epoch 7 with val_acc 0.9166
Epoch 8/100
Train     Loss: 0.1276 | Acc: 0.9532

KeyboardInterrupt: 

In [12]:
#save model
model_save_path = "D:\\Job\\image_retrieval\\models\\covid_cnn_fpn_rmac.pth"
torch.save(model.state_dict(), model_save_path)